In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy import stats
import sys
sys.path.insert(1, '/glade/u/home/lettier/analysis/analysis_antarctic-asym/analysis/')
import asym_funcs as af
xr.set_options(keep_attrs=True)

In [2]:
sy = 1979
ey = 1998

In [3]:
def fix_lp_yr(ds):
    """
    If there are 366 days, interpolate them over 365 days
    For all cases, set time cooridnate to 1 to 365
    """
    if len(ds.time) == 366:
        listda = []
        for var in ds:
            to_interp = ds[var].values
            new_data = np.interp(np.linspace(1,365,365), np.linspace(1,365,366), to_interp)
            newds = xr.DataArray(new_data, dims='time', coords = {'time':np.arange(1,366,1)}).to_dataset(name=var)
            newds.attrs = ds.attrs
            newds[var].attrs = ds[var].attrs
            listda.append(newds)

        newds = xr.merge(listda)
        
    else:
        newds = ds.copy()
        newds['time'] =  np.arange(1,366,1)
            
    return newds

Get CDR observations

In [4]:
mydir = '/glade/work/lettier/CDRv4/day/processed/'

In [5]:
listds = []
for year in np.arange(1979,2019,1):
    ds = xr.open_dataset(mydir+'sia_cdr_daily_nhsh_'+str(year)+'_f17_v04r00.nc')
    ds = fix_lp_yr(ds)
    ds['year'] = year
    ds = ds.set_coords('year')
    listds.append(ds)
ds = xr.concat(listds,dim='year')
ds = ds.interpolate_na(dim='time')
ds.to_netcdf('../processed/sia_sie_daily_CDRv4_1979-2018.nc')

CMIP6 data

In [6]:
mydir = '/glade/work/lettier/CMIP6/historical/processed/day/cmip6_dailysia/'
myfiles = sorted([f for f in os.listdir(mydir) if '.nc' in f])

time_lpyr = xr.open_dataset(mydir+'sia_sie_SIday_historical_ACCESS-CM2_r1i1p1f1_gn_1978-2014.nc').time
time_nolpyr = xr.open_dataset(mydir+'sia_sie_SIday_historical_CNRM-CM6-1_r1i1p1f2_gn_1978-2014.nc').time


listds = []
for f in myfiles:
    
    ds = xr.open_dataset(mydir+f)
    if len(ds.time) == len(time_lpyr): # Make time coordinates consistent across models
        ds['time'] = time_lpyr
    else:
        ds['time'] = time_nolpyr
    if 'type' in ds:
        ds = ds.drop('type')
        
    listyrds = []
    for year in np.arange(sy,ey+1,1): # interpolate data for leap years to 365-day year
        yrds = ds.sel(time=str(year))
        yrds = fix_lp_yr(yrds)
        yrds['year'] = year
        yrds = yrds.set_coords('year')
        yrds['names'] = f.split('_')[4]
        yrds = yrds.set_coords('names')
  
        listyrds.append(yrds)
    ds = xr.concat(listyrds,dim='year')
    ds = ds.mean(dim='year') # make climatology
    listds.append(ds)

ds = xr.concat(listds,dim='names')
ds.attrs['processed'] = 'processed by Lettie Roach, Nov 2021'
print(ds)
ds.to_netcdf('../processed/sia_sie_daily_CMIP6_1979-1998clim.nc')

<xarray.Dataset>
Dimensions:  (time: 365, names: 20)
Coordinates:
  * time     (time) int64 1 2 3 4 5 6 7 8 9 ... 358 359 360 361 362 363 364 365
  * names    (names) <U15 'ACCESS-CM2' 'BCC-CSM2-MR' ... 'SAM0-UNICON'
Data variables:
    sia_nh   (names, time) float64 13.74 13.8 13.87 13.94 ... 14.22 14.26 14.3
    sie_nh   (names, time) float64 14.35 14.42 14.49 14.56 ... 14.89 14.94 14.99
    sia_sh   (names, time) float64 3.765 3.592 3.427 3.267 ... 9.525 9.363 9.203
    sie_sh   (names, time) float64 5.788 5.541 5.312 5.093 ... 14.5 14.33 14.15
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    creation_date:          2020-08-14T05:16:01Z
    ...                     ...
    variant_label:          r1i1p1f1
    version:                v20200814
    license:                CMIP6 model data produced by CSIRO is licensed un...
    cmor_ve

Then CMIP5

In [7]:
mydir = '/glade/work/lettier/CMIP5/historical/processed/day/'
myfiles = sorted([f for f in os.listdir(mydir) if '.nc' in f and 'HadG' not in f])

time_lpyr = xr.open_dataset(mydir+'sia_ACCESS1-0_1980-2005.nc').sel(time=slice('1980',str(ey))).time
time_nolpyr = xr.open_dataset(mydir+'sia_CSIRO-Mk3-6-0_1980-2005.nc').sel(time=slice('1980',str(ey))).time

# Make time indices the same across different models (some include leap days and others do not), set model names as coordinate
listds = []
for f in myfiles:
    ds = xr.open_dataset(mydir+f).sel(time=slice(str(sy),str(ey)))
    if len(ds.time) == len(time_lpyr):
        ds['time'] = time_lpyr
    else:
        ds['time'] = time_nolpyr
    ds['names'] = f.split('_')[1]
    ds = ds.set_coords('names')
    if 'type' in ds:
        ds = ds.drop('type')
    
    listyrds = []
    for year in np.arange(1980,ey+1,1):
        yrds = ds.sel(time=str(year))
        yrds = fix_lp_yr(yrds)
        yrds['year'] = year
        yrds = yrds.set_coords('year')
        listyrds.append(yrds)
    ds = xr.concat(listyrds,dim='year')
    ds = ds.mean(dim='year')/1e6
    listds.append(ds)


ds = xr.concat(listds,dim='names')
ds.attrs = {}
ds.attrs['processed'] = 'processed by Lettie Roach, Nov 2021'
for var in ds:
    ds[var].attrs['units'] = '10^6 km^2'
print(ds)
ds.to_netcdf('../processed/sia_sie_daily_CMIP5_1980-1998clim.nc')

<xarray.Dataset>
Dimensions:  (names: 27, time: 365)
Coordinates:
  * names    (names) <U14 'ACCESS1-0' 'ACCESS1-3' ... 'bcc-csm1-1' 'inmcm4'
  * time     (time) int64 1 2 3 4 5 6 7 8 9 ... 358 359 360 361 362 363 364 365
Data variables:
    sia_nh   (names, time) float64 12.61 12.65 12.7 12.75 ... 12.17 12.23 12.27
    sia_sh   (names, time) float64 3.256 3.124 2.998 2.884 ... 2.198 2.116 2.034
    sie_nh   (names, time) float64 13.01 13.05 13.1 13.15 ... 13.61 13.68 13.75
    sie_sh   (names, time) float64 5.106 4.916 4.733 4.558 ... 3.428 3.293 3.166
Attributes:
    processed:  processed by Lettie Roach, Nov 2021


Compute zonal mean SST from reanalysis

In [8]:
wdir = '/glade/work/lettier/'
sstds = xr.open_dataset(wdir+'NOAA-OISST/sst.mnmean.nc')

def get_zonal_mean_SH_clim(sstds):

    sstds = sstds.assign_coords(lon=(((sstds.lon + 180) % 360) - 180)).sortby('lon') # make longitudes consistent

    # compute zonal mean SST over ocean areas
    lats = sstds.lat.values
    lons = sstds.lon.values
    garea, _, _ = af.grid_area_regll(lats,lons)
    garea = xr.DataArray(-garea,dims=['lat','lon'],coords = {'lat':lats, 'lon':lons}) # compute grid area
    landmask = xr.open_dataset(wdir+'/HADISST/landmask.nc').sftlf
    landmask = landmask.rename({'latitude':'lat','longitude':'lon'})
    oarea = (1.-landmask)*garea # get ocean area
    sst_zonal = (sstds.sst*oarea).sum(dim='lon')/oarea.sum(dim='lon') 
    
    # compute climatology
    sst_zonal_clim = sst_zonal.sel(time=slice('1982','1998')).groupby('time.month').mean(dim='time') 
    
    # drop NH
    sst_zonal_clim_sh = sst_zonal_clim.where(sst_zonal_clim.lat<0,drop=True)
    sst_zonal_clim_sh['lat'] = -sst_zonal_clim_sh.lat
    
    return sst_zonal_clim_sh


def interpolate_monthly_sst_to_daily (obdata):
    
    obdata = obdata.rename({'month':'time'})
    obdata = xr.concat([obdata.isel(time=-1),obdata,obdata.isel(time=0)],dim='time') # choose a random (non leap) year
    obdata['time'] = pd.date_range('15/12/1998',periods=14,freq=pd.DateOffset(months=1))
    obdata = obdata.resample(time='1D').interpolate('linear').sel(time='1999')
    
    obdata['time'] = np.arange(1,366,1)
    shdays = np.append(np.arange(182,366,1),np.arange(1,182,1)) # flip time axis (for SH)
    obdata = obdata.sel(time=shdays)
    obdata['time'] = np.arange(1,366,1)
    obdata = obdata.rename({'time':'day'})
    obdata = obdata.to_dataset(name='sst')
    
    return obdata
    


In [9]:
sst_zonal_clim_sh = get_zonal_mean_SH_clim(sstds)
ds = interpolate_monthly_sst_to_daily (sst_zonal_clim_sh)
ds.to_netcdf('../processed/zonalSST_NOAA-OISST_1982-1998clim.nc')